<a href="https://colab.research.google.com/github/Mohamed-derbeli/Fuel-cell-system/blob/main/Modeling_of_a_Commercial_Heliocentris_PEMFC_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import os
from os import getcwd
import urllib.request
# from urllib.request import urlopen
from urllib.request import urlretrieve
from tensorflow import keras
import codecs

In [ ]:
# with urllib.request.urlopen('https://raw.githubusercontent.com/Mohamed-derbeli/Fuel-cell-system/main/FC50_All_data00.csv') as csvfile:
#    read_data = csvfile.read().decode('utf-8')
#    print(read_data[0:50])

In [107]:
Fan=[]
Temp=[]
I_stack=[]
V_stack=[]
with urllib.request.urlopen('https://raw.githubusercontent.com/Mohamed-derbeli/Fuel-cell-system/main/FC50_All_data00.csv') as csvfile:
    read_file = csv.reader(codecs.iterdecode(csvfile, 'utf-8'), delimiter=',')
    # next(read_file) # next() used to skip the first row 
    rows=[]
    for number in range(0, 60, 6):
      rows.append(number)
    print(rows)
    counter=0    
    for row in read_file:
      if counter in rows:
        V_stack.append(row[0:])
      counter=counter+1
    print(V_stack)
        # labels.append(row[0])
        # sentence=row[1]
        # # for word in stopwords:
        # #     token= " " + word + " " # we used " " to add space before and after the word
        # #     sentence=sentence.replace(token," ") #remove the stopwords from the senctencs
        # #     sentence=sentence.replace("  "," ") #when there are 2 successively changes, they will cause 2 spaces and these should be removed
        # sentences.append(sentence)

[0, 6, 12, 18, 24, 30, 36, 42, 48, 54]
[['6.45', '6.09', '5.34', '4.41', '4.61', '5.52', '6.04', '6.52', '6.93', '7.16', '7.35', '7.49', '7.61', '7.7', '7.79', '7.85', '7.91', '7.97', '8.06', '8.07', '7.95', '7.89', '7.81', '7.73', '7.66', '7.56', '7.42', '7.26', '7.06', '6.81', '6.48', '6.02', '5.39', '4.47', '4.65', '5.56', '6.19', '6.64', '6.96', '7.18', '7.32', '7.47', '7.61', '7.71', '7.79', '7.85', '7.92', '7.97', '8.06', '8.06', '7.95', '7.9', '7.81', '7.74', '7.65', '7.54', '7.43', '7.27', '7.07', '6.83', '6.5', '6.15', '5.41', '4.48', '4.68', '5.58', '6.19', '6.64', '6.95', '7.14', '7.36', '7.5', '7.61', '7.7', '7.79', '7.85', '7.9', '7.97', '8.07', '8.07', '7.97', '7.9', '7.85', '7.75', '7.65', '7.55', '7.43', '7.27', '7.08', '6.82', '6.49', '6.01', '5.38', '4.44', '4.68', '5.57', '6.08', '6.63', '6.95', '7.18', '7.32', '7.47', '7.59', '7.67', '7.75', '7.81', '7.87', '7.93', '8.03', '8.02', '7.93', '7.85', '7.78', '7.72', '7.62', '7.51', '7.38', '7.22', '7.02', '6.83', '6.44'

In [3]:
!gdown "https://github.com/Mohamed-derbeli/Fuel-cell-system/blob/main/FC50_All_data00.csv"
# Get the current working directory
# directory_path = os.getcwd()
# print("Current working directory: ", directory_path)

Downloading...
From: https://github.com/Mohamed-derbeli/Fuel-cell-system/blob/main/FC50_All_data00.csv
To: /content/FC50_All_data00.csv
134kB [00:00, 17.7MB/s]


In [ ]:
 with open('https://github.com/Mohamed-derbeli/Fuel-cell-system/blob/main/FC50_All_data00.csv','r') as csvfile:
   read_data = csv.reader(csvfile, delimiter=',')
#   read_data =  pd.read_excel(exelfile)


#  with open('./FC50_All_data00.csv','r') as csvfile:
# #   read_data =  pd.read_excel(exelfile)
#     read_data = csv.reader(csvfile, delimiter=',')
#     next(read_data)
#     print(read_data)
#     I=[]
#     for row in read_data:
#       I.append(row)
#     print(I)



  # Load the xlsx file
#excel_data = pd.read_excel('./FC50_All_data00.xlsx')
# Read the values of the file in the dataframe
# data = pd.DataFrame(excel_data, columns=[])
# # Print the content
# print("The content of the file is:\n", data)

FileNotFoundError: ignored

In [ ]:
#path= f"{getcwd()}/../tmp2/FC50_All_data00.xlsx"

# Get the current working directory
directory_path = os.getcwd()
print("Current working directory: ", directory_path)
# Change the current working directory
# os.chdir('/tmp')
# print("Current working directory: ", directory_path)

#read_data = open(directory_path) 
# pd.read_excel (path)



Current working directory:  /content


In [ ]:
#Inputs/ Fan: Fan power, Temp: Temperature, I_stack: Stack current, 
#Outputs/ V_stack: Stack voltage

# with open('/tmp/tmpw_yw5r7x') as exelfile:
#     read_data =  pd.read_excel(exelfile)
# read_data = open("https://drive.google.com/drive/search?q=fc50_all_data00")#FC50_All_data00.txt",'r')   
# Fan=[]
# Temp=[]
# I_stack=[]
# V_stack=[]

# for row in read_data:
#   print(row[0])
    #   if row== 1:7:55 
    #     V_stack.append(row[0:])
      # if row[0]=='0':
      #     list_item.append(0)
      #   else:
      #       list_item.append(1)
        ### END CODE HERE
        # num_sentences = num_sentences + 1
        # corpus.append(list_item)

SyntaxError: ignored